# Data Visualization
Visualize the lidar, joystick, and 10m goal data

In [2]:
from dataset import CLIPDataModule, CLIPSet
from termcolor import cprint
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import utils
import os
import glob
%matplotlib inline

## Create CLIPDataModule

In [8]:
dm = CLIPDataModule(data_path='trash',
                    batch_size=32,
                    num_workers=10,
                    joy_len=300,
                    include_lidar_file_names=True,
                    verbose=True)

dm.setup()

loading data from trash...
skip first 50 frames
batch size: 32
future joystick length: 300

creating training set...


  0%|          | 0/1 [00:00<?, ?it/s]

creating validation set...


  0%|          | 0/1 [00:00<?, ?it/s]

training size: 3289 samples
validation size: 2898 samples


In [9]:
cprint(f"number of samples: {len(dm.training_set) + len(dm.validation_set)}",
       'green',
       attrs=['bold'])

number of samples: 6187


## Test Training DataLoader
Check if all batches can be retrieved successfully

### Shape Information:
```lidar_stack.shape``` = (batch size, channels, img height, img width) \
```joystick.shape``` = (batch size, joy_len, 3) \
```goal.shape``` = (batch size, 2)


In [11]:
pickle_files = glob.glob(os.path.join('/home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash', '*.pkl*'))
pickle_files

['/home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash/2021-11-15-14-31-02_final.pkl',
 '/home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash/2021-11-15-14-22-52_final.pkl']

In [13]:
%%time
# check that all samples can be retrieved
for i, f in enumerate(pickle_files):
    cprint(f'pickle_file_path: {f}', 'white', attrs=['bold'])
    d = CLIPSet(pickle_file_path=f)
    for j in range(len(d)):
        try:
            sample = d[j]
        except:
            print(f'failed index: {j}')
    cprint(f'{i + 1}/{len(pickle_files)} completed', 'green', attrs=['bold'])
    

pickle_file_path: /home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash/2021-11-15-14-31-02_final.pkl
failed index: 3282
failed index: 3283
failed index: 3284
failed index: 3285
failed index: 3286
failed index: 3287
failed index: 3288
1/2 completed
pickle_file_path: /home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash/2021-11-15-14-22-52_final.pkl
failed index: 2886
failed index: 2887
failed index: 2888
failed index: 2889
failed index: 2890
failed index: 2891
failed index: 2892
failed index: 2893
failed index: 2894
failed index: 2895
failed index: 2896
failed index: 2897
2/2 completed
CPU times: user 2min 53s, sys: 6.43 s, total: 3min
Wall time: 2min 16s


In [3]:
## 3282
data = pickle.load(open('/home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash/2021-11-15-14-31-02_final.pkl', 'rb'), encoding='latin1')
print(len(data['odom']))
dataset = CLIPSet(
    pickle_file_path='/home/abhinavchadaga/Developer/summer_22_research/clip_social_nav/trash/2021-11-15-14-31-02_final.pkl')
len(dataset)

3332


3282

In [ ]:
# turn train_dataloader into iterable
trainloader = dm.train_dataloader()
train_iter = iter(trainloader)
print(f'number of batches: {len(trainloader)}')

In [ ]:
batch = next(iter(trainloader))
print(type(batch))

## Select Random Sample from Training Data

In [ ]:
import random

# create CLIPset from larger pkl file
train_set = CLIPSet(pickle_file_path='/home/abhinavchadaga/CS/clip_social_nav/data/2021-11-09-12-58-45_final.pkl', include_lidar_file_names=True)

# select random index
idx = random.randint(0, len(train_set))

# get random sample
sample = train_set[idx]
sample_2 = train_set[idx+1]

In [ ]:
lidar = sample[0][0]
lidar.shape

## Visualize Lidar Stack 

In [ ]:
# visualize mode returns lidar stack
# and the corresponding file names
lidar_stack = sample[0][0]
lidar_fnames = sample[0][1]

cprint(f'index: {idx}, total offset: 50', color='green', attrs=['bold'])
utils.visualize_lidar_stack(lidar_stack=lidar_stack, file_names=lidar_fnames)

## Visualize Joystick Data

In table format

In [ ]:
import numpy as np

joystick = sample[1]
print('joystick dtype: ', joystick.dtype)
print('joystick shape: ', joystick.shape)

joy_df = pd.DataFrame(data=joystick,
                      columns=['linear_x', 'linear_y', 'angular_z'])
joy_df

In [ ]:
joy_df_2 = pd.DataFrame(data=sample_2[1],
                      columns=['linear_x', 'linear_y', 'angular_z'])
joy_df_2


In [ ]:
pd.set_option('display.max_rows', 500)

## Goal Information
(x, y) location relative to the current position of the robot

In [ ]:
goal = sample[2]
print(f'x: {goal[0]:.2f}, y: {goal[1]:.2f}')

In [ ]:
import numpy as np

data = pickle.load(open('../data/2021-11-15-15-06-15_final.pkl', 'rb'), encoding='latin1')
lidar_frame_w_goal = utils.visualize_goal(lidar_frame=lidar_stack[-1], goal=goal)
plt.figure(figsize=(10, 10))
plt.imshow(lidar_frame_w_goal);